In [1]:
import pandas as pd
import numpy as np
from os.path import join
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from ete3 import NCBITaxa
import random
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
random.seed(10)
import torch
import esm
from bioservices import *
from data_preprocessing import *
from functions_and_dicts_data_preprocessing_GNN import *
from build_GNN import *
import warnings
warnings.filterwarnings('ignore')
datasets_dir = "../../data"

CURRENT_DIR = os.getcwd()

c:\Users\marle\anaconda3\envs\py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Loading in Sabio data

#### Loading Sabio data

In [2]:
organism = "Seed plants"

df_Sabio = pd.read_table(join("..", "..", "data", "Km_model_" + organism + ".tsv"))

df_Sabio["Km"] = df_Sabio["Km"].astype('float')
df_Sabio["PMID"] = df_Sabio["PMID"].astype('Int64')

df_Sabio["substrate_IDs"] = df_Sabio["substrate_IDs"].str.split('#')
df_Sabio["product_IDs"] = df_Sabio["product_IDs"].str.split('#')

print("Number of data points: %s" % len(df_Sabio))
print("Number of UniProt IDs: %s" % len(set(df_Sabio["Uniprot IDs"])))

df_Km = df_Sabio

Number of data points: 3635
Number of UniProt IDs: 835


#### Removing duplicates

In [ ]:
droplist = []

for ind in df_Km.index:
    UID, Km = df_Km["Uniprot IDs"][ind], df_Km["Km"][ind]
    help_df = df_Km.loc[df_Km["Uniprot IDs"] == UID].loc[df_Km["Km"] == Km]
    
    if len(help_df) > 1:
        droplist = droplist + list(help_df.index)[1:]
        

In [ ]:
df_Km.drop(list(set(droplist)), inplace = True)
print("Dropping %s data points, because they are duplicated." % len(set(droplist)))
df_Km.reset_index(inplace = True, drop = True)
df_Km

#### Removing top and bottom 3% of Km values

In [ ]:
def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

find_outliers_IQR(df_Km["Km"])

print(df_Km['Km'].quantile(0.03),  df_Km['Km'].quantile(0.97))

In [ ]:
df_Km = df_Km[(df_Km['Km'] > df_Km['Km'].quantile(0.03)) & (df_Km['Km'] < df_Km['Km'].quantile(0.97))]
df_Km.reset_index(inplace = True, drop = True)

In [ ]:
todrop= []

for ind in df_Km.index:
    UID = df_Km["Uniprot IDs"][ind]
    if len(UID.split(';')) > 1:
        todrop.append(ind)
        print(df_Km["Uniprot IDs"][ind])
        print(todrop)
        
df_Km.drop(todrop, inplace=True)
df_Km.reset_index(inplace = True, drop = True)

In [ ]:
df_Km["substrate_IDs"] = df_Km["substrate_IDs"].apply(lambda x: (set(x)))
df_Km["product_IDs"] = df_Km["product_IDs"].apply(lambda x: (set(x)))

In [ ]:
df_Km.to_pickle(join("..", "..", "data", "Km_data_merged.pkl"))

## 2. Assigning IDs to every unique sequence and to every unique reaction in the dataset

#### Creating DataFrames for all sequences and for all reactions

In [ ]:
df_reactions = pd.DataFrame({"substrates": df_Km["substrate_IDs"],
                            "products" : df_Km["product_IDs"]})

df_reactions = df_reactions.loc[df_reactions["substrates"] != set([])]
df_reactions = df_reactions.loc[df_reactions["products"] != set([])]


droplist = []
for ind in df_reactions.index:
    sub_IDs, pro_IDs = df_reactions["substrates"][ind], df_reactions["products"][ind]
    help_df = df_reactions.loc[df_reactions["substrates"] == sub_IDs].loc[df_reactions["products"] == pro_IDs]
    if len(help_df):
        for ind in list(help_df.index)[1:]:
            droplist.append(ind)
            
df_reactions.drop(list(set(droplist)), inplace = True)
df_reactions.reset_index(inplace = True, drop =True)

df_reactions["Reaction ID"] = ["Reaction_" + str(ind) for ind in df_reactions.index]

In [ ]:
df_sequences = pd.DataFrame(data = {"Sequence" : df_Km["Sequence"].unique()})
df_sequences = df_sequences.loc[~pd.isnull(df_sequences["Sequence"])]
df_sequences.reset_index(inplace = True, drop = True)
df_sequences["Sequence ID"] = ["Sequence_" + str(ind) for ind in df_sequences.index]

df_sequences

#### Calculating minimal Km value for each reaction and sequence

In [ ]:
df_reactions["min_Km_for_RID"] = np.nan
for ind in df_reactions.index:
    df_reactions["min_Km_for_RID"][ind] = min(df_Km.loc[df_Km["substrate_IDs"] == df_reactions["substrates"][ind]].loc[df_Km["product_IDs"] == df_reactions["products"][ind]]["Km"])

In [ ]:
df_sequences["min_Km_for_UID"] = np.nan
for ind in df_sequences.index:
    df_sequences["min_Km_for_UID"][ind] = min(df_Km.loc[df_Km["Sequence"] == df_sequences['Sequence'][ind]]["Km"])

#### Calculating the sum of the molecular weights of all substrates and of all products

In [ ]:
df_reactions["MW_frac"] = np.nan

for ind in df_reactions.index:
    substrates = list(df_reactions["substrates"][ind])
    products = list(df_reactions["products"][ind])
    
    mw_subs = mw_mets(metabolites = substrates)
    mw_pros = mw_mets(metabolites = products)
    
    if mw_subs == np.nan or mw_pros == np.nan:
        df_reactions["MW_frac"][ind] = np.inf
    if mw_pros != 0:
        df_reactions["MW_frac"][ind] = mw_subs/mw_pros
    else:
        df_reactions["MW_frac"][ind] = np.inf
        
df_reactions

#### Calculating enzyme, reaction and substrate features

In [ ]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")

In [ ]:
#creating model input:
df_sequences["model_input"] = [seq[:1022] for seq in df_sequences["Sequence"]]
model_input = [(df_sequences["Sequence ID"][ind], df_sequences["model_input"][ind]) for ind in df_sequences.index]
seqs = [model_input[i][1] for i in range(len(model_input))]
#loading ESM-2 model:
print(".....2(a) Loading ESM-2 model.")
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
#convert input into batches:

#Calculate ESM-2 representations
print(".....2(b) Calculating enzyme representations.")
df_sequences["Enzyme rep"] = ""

for ind in df_sequences.index:
    batch_labels, batch_strs, batch_tokens = batch_converter([(df_sequences["Sequence ID"][ind], df_sequences["model_input"][ind])])
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])
    df_sequences["Enzyme rep"][ind] = results["representations"][33][0, 1 : len(df_sequences["model_input"][ind]) + 1].mean(0).numpy()
    
df_sequences.head(5)

In [ ]:
def get_metabolite_type(met):
    if is_KEGG_ID(met):
        return("KEGG")
    elif is_InChI(met):
        return("InChI")
    else:
        return("invalid")

def get_reaction_site_smarts(metabolites):
    reaction_site = ""
    for met in metabolites:
        met_type = get_metabolite_type(met)
        if met_type == "KEGG":
            try:
                Smarts = Chem.MolToSmarts(Chem.MolFromMolFile(join(CURRENT_DIR, "data", "mol-files",  met + ".mol")))
            except OSError:
                return(np.nan)
        elif met_type == "InChI":
            Smarts = Chem.MolToSmarts(Chem.inchi.MolFromInchi(met))
        else:
            Smarts = "invalid"
        reaction_site = reaction_site + "." + Smarts
    return(reaction_site[1:])


def is_KEGG_ID(met):
    #a valid KEGG ID starts with a "C" or "D" followed by a 5 digit number:
    if len(met) == 6 and met[0] in ["C", "D"]:
        try:
            int(met[1:])
            return(True)
        except: 
            pass
    return(False)

def is_InChI(met):
    m = Chem.inchi.MolFromInchi(met,sanitize=False)
    if m is None:
      return(False)
    else:
      try:
        Chem.SanitizeMol(m)
      except:
        print('.......Metabolite string "%s" is in InChI format but has invalid chemistry' % met)
        return(False)
    return(True)

def convert_fp_to_array(difference_fp_dict):
    fp = np.zeros(2048)
    for key in difference_fp_dict.keys():
        fp[key] = difference_fp_dict[key]
    return(fp)

In [ ]:
df_reactions["difference_fp"], df_reactions["structural_fp"],  = "", ""
#each metabolite should be either a KEGG ID, InChI string, or a SMILES:
for ind in df_reactions.index:
    left_site = get_reaction_site_smarts(df_reactions["substrates"][ind])
    right_site = get_reaction_site_smarts(df_reactions["products"][ind])
    if not pd.isnull(left_site) and not pd.isnull(right_site):
        rxn_forward = AllChem.ReactionFromSmarts(left_site + ">>" + right_site)
        difference_fp = Chem.rdChemReactions.CreateDifferenceFingerprintForReaction(rxn_forward)
        difference_fp = convert_fp_to_array(difference_fp.GetNonzeroElements())
        df_reactions["difference_fp"][ind] = difference_fp
        df_reactions["structural_fp"][ind] = Chem.rdChemReactions.CreateStructuralFingerprintForReaction(rxn_forward).ToBitString()

df_reactions.head(5)

In [ ]:
df_sequences.to_pickle(join(datasets_dir, "all_sequences_with_IDs.pkl"))
df_reactions.to_pickle(join(datasets_dir, "all_reactions_with_IDs.pkl"))

#### Mapping Sequence and Reaction IDs to Km_df

In [ ]:
df_Km = df_Km.merge(df_sequences, on = "Sequence", how = "left")

In [ ]:
df_reactions.rename(columns = {"substrates" : "substrate_IDs",
                              "products" : "product_IDs"}, inplace = True)

df_Km["Reaction ID"] = np.nan
df_Km["MW_frac"] = np.nan
df_Km["min_Km_for_RID"] = np.nan
df_Km["difference_fp"] = ""
df_Km["structural_fp"] = ""

for ind in df_Km.index:
    sub_set, pro_set = df_Km["substrate_IDs"][ind], df_Km["product_IDs"][ind]
    help_df = df_reactions.loc[df_reactions["substrate_IDs"] == sub_set].loc[df_reactions["product_IDs"] == pro_set]
    if len(help_df) == 1:
        df_Km["Reaction ID"][ind] = list(help_df["Reaction ID"])[0]
        df_Km["min_Km_for_RID"][ind] = list(help_df["min_Km_for_RID"])[0]
        df_Km["MW_frac"][ind] = list(help_df["MW_frac"])[0]
        df_Km["difference_fp"][ind] = list(help_df["difference_fp"])[0]
        df_Km["structural_fp"][ind] = list(help_df["structural_fp"])[0]
df_Km.head(2)

In [ ]:
df_Km["MACCS FP"] = ""

for ind in df_Km.index:
    substrate = df_Km["Main substrate"][ind]
    try:
        id = df_Km['Substrate_IDs'][ind][df_Km["Substrates"][ind].split(';').index(substrate)]
    except:
        for i,s in enumerate(df_Km["Substrates"][ind].split(';')[:-1]):
            if substrate in s or s in substrate:
                id = list(df_Km['Substrate_IDs'][ind])[i]
    if id[0] == "C":
        try:
            mol = Chem.MolFromMolFile(join(datasets_dir,"mol-files", id + '.mol'))
        except OSError:
            None
    else:
        try:
            mol = Chem.inchi.MolFromInchi(id,sanitize=False)
        except OSError:
            None
    if mol is not None:
        maccs_fp = MACCSkeys.GenMACCSKeys(mol).ToBitString()
        df_Km["MACCS FP"][ind] = maccs_fp

#### Calculating the minimal Km value for every EC number in the dataset

In [ ]:
df_EC_Km = pd.read_csv(join("..", "..", "data", "min_EC_" + organism + ".tsv"), sep = "\t", header=0)
# df_EC_Km = df_EC_Km.rename(columns={0: "EC", 1: "min_Km"})

for ind in df_EC_Km.index:
    try:
        Km_min = df_EC_Km[df_EC_Km["EC"] == df_Km["ECs"]]["min_Km"]
        df_EC_Km["min_Km"][ind] = Km_min
        print(ind, Km_min)
    except:
        pass
        
df_EC_Km.describe()

In [ ]:
df_EC_Km = pd.read_csv(join("..", "..", "data", "min_EC_" + organism + ".tsv"), sep = "\t", header=0)

df_EC_Km.head(5)
df_Km["min_Km_for_EC"] = np.nan

for ind in df_Km.index:
    EC = df_Km["ECs"][ind]
    min_Km = 0
    try:
        print(EC)
        min_Km = df_EC_Km.loc[df_EC_Km["EC"] == EC, "min_Km"].iloc[0]
        print(min_Km)
    except:
        pass
    if min_Km != 0:
        df_Km["min_Km_for_EC"][ind] = min_Km
df_Km.to_pickle(join("..", "..", "data", "merged_and_grouped_Km_dataset.pkl"))  

## 3. Removing outliers

#### Removing non-optimally measured values

To ignore $Km$ values that were obtained under non-optimal conditions, we exclude values higher than 10000\% than the minimal $Km$ value for the same enzyme-reaction combination.

In [ ]:
df_Km["frac_of_min_UID"] = np.nan
df_Km["frac_of_min_RID"] = np.nan
df_Km["frac_of_min_EC"] = np.nan

for ind in df_Km.index:
    df_Km["frac_of_min_UID"][ind] =  df_Km["min_Km_for_UID"][ind]/df_Km["Km"][ind]
    df_Km["frac_of_min_RID"][ind] =  df_Km["min_Km_for_RID"][ind]/df_Km["Km"][ind]
    df_Km["frac_of_min_EC"][ind] = df_Km["min_Km_for_EC"][ind]/df_Km["Km"][ind]

len(df_Km)

In [ ]:
n = len(df_Km)

df_Km = df_Km.loc[df_Km["frac_of_min_UID"] >= 0.01]
df_Km = df_Km.loc[df_Km["frac_of_min_RID"] >= 0.01]

df_Km["frac_of_min_EC"].loc[pd.isnull(df_Km["frac_of_min_EC"])] = 1
df_Km = df_Km.loc[df_Km["frac_of_min_EC"] <= 10]
df_Km = df_Km.loc[df_Km["frac_of_min_EC"] >= 0.01]

In [ ]:
print("We remove %s data points, because we suspect that these Km values were not measure for the natural reaction " \
    "of an enzyme or under non-optimal conditions." % (n-len(df_Km)))

#### Removing data points with reaction queations with uneven fraction of molecular weights

In [ ]:
n = len(df_Km)

df_Km = df_Km.loc[df_Km["MW_frac"] < 3]
df_Km = df_Km.loc[df_Km["MW_frac"] > 1/3]

print("We remove %s data points because the sum of molecular weights of substrates does not match the sum of molecular" \
      "weights of the products." % (n-len(df_Km)))

In [ ]:
print("Size of final Km dataset: %s" % len(df_Km))
df_Km.to_pickle(join("..", "..", "data", "final_Km_dataset_" + organism + ".pkl"))

## 4. Preparing dataset and splitting into train-test

In [3]:
df_Km = pd.read_pickle(join("..", "..", "data", "final_Km_dataset_" + organism + ".pkl"))

#### Splitting glucosinolates into validation dataset

Search UniProt for GO term related to glucosionalte metabolic process, download file as .tsv and filter dataset

In [ ]:
# glucosinolates = pd.read_table(join(datasets_dir,"glucosinolates.tsv"))["Entry"].tolist()
# df_Km = df_Km[~df_Km["Uniprot IDs"].isin(glucosinolates)]
# df_validation = df_Km[df_Km["Uniprot IDs"].isin(glucosinolates)]

If training-testing with only Arabidopsis data:

In [ ]:
# df_Km = df_Km[df_Km["Organism"] == 'Arabidopsis thaliana']

If training-testing with only Brassicaceae data:

In [ ]:
# ncbi = NCBITaxa()

# organisms = {}

# def is_brassicaceae(org):
#     try:
#         tax_id = ncbi.get_name_translator([org])[org][0]
#         lineage = ncbi.get_lineage(tax_id)
#         if 3700 not in lineage:
#             return(False)
#         else:
#             return(True)
#     except KeyError:
#         return(False)
    
# for org in df_Km["Organism"].tolist():
#     if org not in organisms.keys():
#         organisms[org] = is_brassicaceae(org)

# df_Km = df_Km[df_Km["Organism"].isin([key for key, value in organisms.items() if value is True])]
# df_Km.reset_index(inplace=True, drop = True)

If training-testing only with wildtype data:

In [ ]:
# df_Km = df_Km[df_Km["Type"] == "wildtype"]

#### Calculating arithmetic mean for Km values of same enzyme-reaction-substrate combination-pH-temperature

In [ ]:
# df_new = pd.DataFrame(data = {"Reaction ID" : df_Km["Reaction ID"],
#                                   "Sequence ID" : df_Km["Sequence ID"],
#                                   "Temperature" : df_Km["Temperature"],
#                                     "pH" : df_Km["pH"],
#                                  "Type": df_Km["Type"],
#                              "MACCS FP" : df_Km["MACCS FP"]})

# df_new.drop_duplicates(inplace = True)
# df_new.reset_index(inplace = True, drop = True)

# df_new["Km_values"], df_new["Uniprot IDs"], df_new["ECs"], df_new["Substrates"], df_new["Products"], df_new["ESM2"], df_new["Sequence"], df_new["difference_fp"], df_new["structural_fp"] = "", "", "", "", "", "", "", "", ""

# for ind in df_new.index:
#     RID, SID, Temp, pH, Type, MSubstrate = df_new["Reaction ID"][ind], df_new["Sequence ID"][ind], df_new["Temperature"][ind], df_new["pH"][ind], df_new["Type"][ind], df_new["MACCS FP"][ind]
#     help_df = df_Km.loc[df_Km["Reaction ID"] 
#                                  == RID].loc[df_Km["Sequence ID"] 
#                                              == SID].loc[df_Km["Temperature"] 
#                                                          == Temp].loc[df_Km["pH"] 
#                                                                       == pH].loc[df_Km["Type"] 
#                                                                                  == Type].loc[df_Km["MACCS FP"] 
#                                                                                               == MSubstrate]
#     print(help_df)
#     df_new["ECs"][ind] = list(help_df["ECs"])
#     df_new["Km_values"][ind] = list(help_df["Km"])
#     df_new["Uniprot IDs"][ind] = list(help_df["Uniprot IDs"])
#     df_new["Sequence"][ind] = help_df["Sequence"].values[0]
#     df_new["ESM2"][ind] = help_df["Enzyme rep"].values[0]
#     df_new["difference_fp"][ind], df_new["structural_fp"][ind] = help_df["difference_fp"].values[0], help_df["structural_fp"].values[0]
#     df_new["Substrates"][ind], df_new["Products"][ind] = help_df["Substrates"].values[0], help_df["Products"].values[0]

In [ ]:
# df_new2 = pd.DataFrame(data = {"Reaction ID" : df_validation["Reaction ID"],
#                                   "Sequence ID" : df_validation["Sequence ID"],
#                                   "Temperature" : df_validation["Temperature"],
#                                     "pH" : df_validation["pH"],
#                                   "Type" : df_validation["Type"],
#                                   "MACCS FP" : df_validation["MACCS FP"]})

# df_new2.drop_duplicates(inplace = True)
# df_new2.reset_index(inplace = True, drop = True)

# df_new2["Km_values"], df_new2["Uniprot IDs"], df_new2["ECs"], df_new2["Organisms"], df_new2["Substrates"], df_new2["Products"], df_new2["ESM2"], df_new2["Sequence"], df_new2["difference_fp"], df_new2["structural_fp"] = "", "", "", "", "", "", "", "", "", ""

# for ind in df_new2.index:
#     RID, SID, Temp, pH, Type, MSubstrate = df_new2["Reaction ID"][ind], df_new2["Sequence ID"][ind], df_new2["Temperature"][ind], df_new2["pH"][ind], df_new2["Type"][ind], df_new2["MACCS FP"][ind]
#     help_df = df_validation.loc[df_validation["Reaction ID"] 
#                               == RID].loc[df_validation["Sequence ID"] 
#                                           == SID].loc[df_validation["Temperature"] 
#                                                       == Temp].loc[df_validation["pH"] 
#                                                                     == pH].loc[df_validation["Type"] 
#                                                                               == Type].loc[df_validation["MACCS FP"] 
#                                                                                                             == MSubstrate]
#     df_new2["ECs"][ind] = list(help_df["ECs"])
#     df_new2["Km_values"][ind] = list(help_df["Km"])
#     df_new2["Uniprot IDs"][ind] = list(help_df["Uniprot IDs"])
#     df_new2["Organisms"][ind] = list(help_df["Organism"])
#     df_new2["Type"][ind]
#     df_new2["Sequence"][ind] = help_df["Sequence"].values[0]
#     df_new2["ESM2"][ind] = help_df["Enzyme rep"].values[0]
#     df_new2["difference_fp"][ind], df_new2["structural_fp"][ind] = help_df["difference_fp"].values[0], help_df["structural_fp"].values[0]
#     df_new2["Substrates"][ind], df_new2["Products"][ind] = help_df["Substrates"].values[0], help_df["Products"].values[0]

In [ ]:
# df_Km = df_new
# df_validation = df_new2

In [ ]:
# df_Km["geomean_Km"] = np.nan
# for ind in df_Km.index:
#     all_Km = np.array(df_Km["Km_values"][ind]).astype(float)
    # min_Km = min(all_Km)
    # all_Km_top = [Km for Km in all_Km  if min_Km/Km >= 0.01]
    # df_arabidopsis["geomean_Km"][ind] = np.mean((all_Km_top))

In [ ]:
# df_validation["geomean_Km"] = np.nan
# for ind in df_validation.index:
#     all_Km = np.array(df_validation["Km_values"][ind]).astype(float)
#     min_Km = min(all_Km)
#     all_Km_top = [Km for Km in all_Km  if min_Km/Km >= 0.01]
#     df_validation["geomean_Km"][ind] = np.mean((all_Km_top))
    
# df_validation.to_pickle(join("..", "..", "data", "validation_%s.pkl" %organism))


#### Making input for GNN

In [4]:
for ind in df_Km.index:
    substrate = df_Km["Main Substrate"][ind]
    try:
        id = list(df_Km['substrate_IDs'][ind])[df_Km["Substrates"][ind].split(';').index(substrate)]
    except:
        for i,s in enumerate(df_Km["Substrates"][ind].split(';')[:-1]):
            if substrate in s or s in substrate:
                id = list(df_Km['substrate_IDs'][ind])[i]
    df_Km["Main Substrate"][ind] = id

inchi_ids = {}
for i, element in enumerate(df_Km["Main Substrate"]):
    if element[0] != 'C' and element not in inchi_ids.keys():
        inchi_ids[element] = str(i)
        mol = Chem.inchi.MolFromInchi(element)
        if not mol is None:
            calculate_atom_and_bond_feature_vectors(mol, str(i))
        # Chem.rdmolfiles.MolToMolFile(Chem.inchi.MolFromInchi(element), join(datasets_dir,"mol-files", str(i) + ".mol")  )  

#### Splitting into train-test

In [5]:
df = df_Km.copy()
df = df.sample(frac = 1, random_state = 123)
df.reset_index(drop= True, inplace = True)

train_df, test_df = split_dataframe_enzyme(frac = 5, df = df.copy())
print("Test set size: %s" % len(test_df))
print("Training set size: %s" % len(train_df))
print("Size of test set in percent: %s" % np.round(100*len(test_df)/ (len(test_df) + len(train_df))))

train_df.reset_index(inplace = True, drop = True)
test_df.reset_index(inplace = True, drop = True)

train_df.to_pickle(join(datasets_dir, "train_df_Km_%s.pkl" %organism))
test_df.to_pickle(join(datasets_dir, "test_df_Km_%s.pkl" %organism))

Test set size: 503
Training set size: 1644
Size of test set in percent: 23.0


#### Splitting CV folds

In [6]:
data_train2 = train_df.copy()
data_train2["index"] = list(data_train2.index)

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=5)
indices_fold1 = list(df_fold["index"])
print(len(data_train2), len(indices_fold1))#

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=4)
indices_fold2 = list(df_fold["index"])
print(len(data_train2), len(indices_fold2))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=3)
indices_fold3 = list(df_fold["index"])
print(len(data_train2), len(indices_fold3))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=2)
indices_fold4 = list(df_fold["index"])
indices_fold5 = list(data_train2["index"])
print(len(data_train2), len(indices_fold4))


fold_indices = [indices_fold1, indices_fold2, indices_fold3, indices_fold4, indices_fold5]

CV_train_indices = [[], [], [], [], []]
CV_test_indices = [[], [], [], [], []]

for i in range(5):
    for j in range(5):
        if i != j:
            CV_train_indices[i] = CV_train_indices[i] + fold_indices[j]
    CV_test_indices[i] = fold_indices[i]
    
    
np.save(join(datasets_dir, "CV_train_indices_%s" %organism), CV_train_indices)
np.save(join(datasets_dir, "CV_test_indices_%s" %organism), CV_test_indices)

1336 308
969 367
671 298
322 349


## 5. Building GNN for substrate representation

In [7]:
for ind in train_df.index:
    calculate_and_save_input_matrixes(inchi_ids, sample_ID = "train_" + str(ind), df = train_df,
                                      save_folder = join(datasets_dir, "GNN_input_data"))
    
for ind in test_df.index:
    calculate_and_save_input_matrixes(inchi_ids, sample_ID = "test_" + str(ind), df = test_df,
                                      save_folder = join(datasets_dir, "GNN_input_data"))

Could not create input for substrate ID 1542


In [8]:
train_indices = os.listdir(join(datasets_dir, "GNN_input_data"))
train_indices = [index[:index.rfind("_")] for index in train_indices]
train_indices = list(set([index for index in train_indices if "train" in index]))

test_indices = os.listdir(join(datasets_dir, "GNN_input_data"))
test_indices = [index[:index.rfind("_")] for index in test_indices]
test_indices = list(set([index for index in test_indices if "test" in index]))

#### Hyper-parameter optimization with CV

In [9]:
# optimization = pd.DataFrame()

param_grid = {'batch_size': [64,96,128],
                'D': [50,100],
                'learning_rate': [0.01, 0.1],
                'epochs': [30,50,80],
                'l2_reg_fc' : [0.01, 0.1, 1],
                'l2_reg_conv': [0.01, 0.1, 1],
                'rho': [0.9, 0.95, 0.99]}

params_list = [(batch_size, D, learning_rate, epochs, l2_reg_fc, l2_reg_conv, rho) for batch_size in param_grid['batch_size'] for D in param_grid["D"] for learning_rate in param_grid['learning_rate']
                for epochs in param_grid['epochs'] for l2_reg_fc in param_grid['l2_reg_fc'] for l2_reg_conv in param_grid['l2_reg_conv'] for rho in param_grid["rho"]]

params_list = random.sample(params_list, 10)

In [10]:
count = 0
results=[]

for params in params_list:

    batch_size, D, learning_rate, epochs, l2_reg_fc, l2_reg_conv, rho = params
    count +=1
    MAE = []

    for i in range(5):
        train_index, test_index  = CV_train_indices[i], CV_test_indices[i]
        train_index = [ind for ind in train_indices if int(ind.split("_")[1]) in train_index]
        test_index = [ind for ind in train_indices if int(ind.split("_")[1]) in test_index]

        train_params = {'batch_size': batch_size,
                    'folder' :join(datasets_dir, "GNN_input_data"),
                    'list_IDs' : np.array(train_index),
                    'shuffle': True}

        test_params = {'batch_size': min(batch_size,len(test_index)),
                    'folder' : join(datasets_dir, "GNN_input_data"),
                    'list_IDs' : np.array(test_index),
                    'shuffle': False}

        training_generator = DataGenerator(**train_params)
        test_generator = DataGenerator(**test_params)


        model = DMPNN_without_extra_features(l2_reg_conv = l2_reg_conv, l2_reg_fc = l2_reg_fc, learning_rate = learning_rate,
                        D = D, N = N, F1 = F1, F2 = F2, F= F, drop_rate = 0.0, ada_rho = rho)
        model.fit(training_generator, epochs= epochs, shuffle = True, verbose = 1)

        #get test_y:
        test_indices_y = [int(ind.split("_")[1]) for ind in train_indices if ind in test_index]
        test_y = np.array([train_df["Km"][ind] for ind in test_indices_y])

        pred_test = model.predict(test_generator)
        mae = np.median(abs(pred_test - np.reshape(test_y[:len(pred_test)], (-1,1))))
        print(mae)
        MAE.append(mae)

    results.append({"batch_size" : batch_size, "D" : D , "learning_rate" : learning_rate, "epochs" : epochs,
                    "l2_reg_fc" : l2_reg_fc, "l2_reg_conv" : l2_reg_conv, "rho" : rho, "cv_mae" : np.mean(MAE)})

params = min(results, key=lambda d: d['cv_mae'])
print(params)

Epoch 1/30
11/11 [==============================] - 40s 3s/step - loss: 5.2349 - mse: 0.0268 - mae: 0.1362
Epoch 2/30
11/11 [==============================] - 12s 978ms/step - loss: 5.0460 - mse: 0.0168 - mae: 0.1088
Epoch 3/30
11/11 [==============================] - 11s 900ms/step - loss: 4.8628 - mse: 0.0141 - mae: 0.1009
Epoch 4/30
11/11 [==============================] - 10s 804ms/step - loss: 4.6823 - mse: 0.0128 - mae: 0.0963
Epoch 5/30
11/11 [==============================] - 10s 803ms/step - loss: 4.5043 - mse: 0.0116 - mae: 0.0938
Epoch 6/30
11/11 [==============================] - 10s 808ms/step - loss: 4.3296 - mse: 0.0111 - mae: 0.0914
Epoch 7/30
11/11 [==============================] - 10s 796ms/step - loss: 4.1579 - mse: 0.0103 - mae: 0.0888
Epoch 8/30
11/11 [==============================] - 10s 797ms/step - loss: 3.9892 - mse: 0.0094 - mae: 0.0844
Epoch 9/30
11/11 [==============================] - 10s 850ms/step - loss: 3.8248 - mse: 0.0091 - mae: 0.0827
Epoch 10/30
1

{'batch_size': 96, 'D': 50, 'learning_rate': 0.1, 'epochs': 80, 'l2_reg_fc': 0.1, 'l2_reg_conv': 0.1, 'rho': 0.99, 'cv_mae': 0.005500673513143063}

#### Training the model with the best set of hyperparmeters on the whole training set and validate it on the test set

In [11]:
batch_size = 96
D = 50
learning_rate = 0.1
epochs = 80
l2_reg_fc = 0.1
l2_reg_conv = 0.1
rho = 0.99

In [12]:
train_indices = os.listdir(join(datasets_dir, "GNN_input_data"))
train_indices = [index[:index.rfind("_")] for index in train_indices]
train_indices = list(set([index for index in train_indices if "train" in index]))

test_indices = os.listdir(join(datasets_dir, "GNN_input_data"))
test_indices = [index[:index.rfind("_")] for index in test_indices]
test_indices = list(set([index for index in test_indices if "test" in index]))


train_params = {'batch_size': batch_size,
              'folder' :join(datasets_dir, "GNN_input_data"),
              'list_IDs' : train_indices,
              'shuffle': True}

test_params = {'batch_size': batch_size,
              'folder' :join(datasets_dir, "GNN_input_data"),
              'list_IDs' : test_indices,
              'shuffle': False}

training_generator = DataGenerator(**train_params)
test_generator = DataGenerator(**test_params)

model = DMPNN_without_extra_features(l2_reg_conv = l2_reg_conv, l2_reg_fc = l2_reg_fc, learning_rate = learning_rate,
                  D = D, N = N, F1 = F1, F2 = F2, F= F, drop_rate = 0.0, ada_rho = rho)

model.fit(training_generator, epochs= epochs, shuffle = True, verbose = 1)
model.save_weights(join(datasets_dir, "model_weights", "saved_model_GNN_best_hyperparameters"))

pred_test = model.predict(test_generator)
test_indices_y = [int(ind.split("_")[1]) for ind in np.array(test_indices)]
test_y = np.array([test_df["Km"][ind] for ind in test_indices_y])

Epoch 1/80
14/14 [==============================] - 34s 2s/step - loss: 1.6208 - mse: 0.0235 - mae: 0.1227
Epoch 2/80
14/14 [==============================] - 14s 960ms/step - loss: 1.4804 - mse: 0.0124 - mae: 0.0958
Epoch 3/80
14/14 [==============================] - 14s 955ms/step - loss: 1.3602 - mse: 0.0097 - mae: 0.0862
Epoch 4/80
14/14 [==============================] - 14s 964ms/step - loss: 1.2529 - mse: 0.0083 - mae: 0.0792
Epoch 5/80
14/14 [==============================] - 14s 968ms/step - loss: 1.1560 - mse: 0.0071 - mae: 0.0730
Epoch 6/80
14/14 [==============================] - 14s 918ms/step - loss: 1.0686 - mse: 0.0063 - mae: 0.0687
Epoch 7/80
14/14 [==============================] - 22s 1s/step - loss: 0.9896 - mse: 0.0055 - mae: 0.0633
Epoch 8/80
14/14 [==============================] - 14s 955ms/step - loss: 0.9179 - mse: 0.0048 - mae: 0.0590
Epoch 9/80
14/14 [==============================] - 15s 982ms/step - loss: 0.8527 - mse: 0.0042 - mae: 0.0551
Epoch 10/80
14/1

#### Calculating substrate representation for every data point in training and test set

In [13]:
model = DMPNN_without_extra_features(l2_reg_conv = l2_reg_conv, l2_reg_fc = l2_reg_fc, learning_rate = learning_rate,
                  D = D, N = N, F1 = F1, F2 = F2, F= F, drop_rate = 0.0, ada_rho = rho)
model.load_weights(join(datasets_dir, "model_weights", "saved_model_GNN_best_hyperparameters"))

get_fingerprint_fct = K.function([model.layers[0].input, model.layers[26].input,
                                  model.layers[3].input],
                                  [model.layers[-10].output])

In [14]:
input_data_folder = join("../../data/GNN_input_data")   

def get_representation_input(cid_list):
    XE = ();
    X = ();
    A = ();
    # Generate data
    for cid in cid_list:
        try:
            X = X + (np.load(join(input_data_folder, cid + '_X.npy')), );
            XE = XE + (np.load(join(input_data_folder, cid + '_XE.npy')), );
            A = A + (np.load(join(input_data_folder, cid + '_A.npy')), );
        except FileNotFoundError: #return zero arrays:
            X = X + (np.zeros((N,32)), );
            XE = XE + (np.zeros((N,N,F)), );
            A = A + (np.zeros((N,N,1)), );
    return(XE, X, A)

input_data_folder = join("../../data/GNN_input_data") 
def get_substrate_representations(df, training_set, get_fingerprint_fct):
    df["GNN FP"] = ""
    i = 0
    n = len(df)
    
    cid_all = list(df.index)
    if training_set == True:
        prefix = "train_"
    else:
        prefix = "test_"
    cid_all = [prefix + str(cid) for cid in cid_all]
    
    while i*64 <= n:
        if (i+1)*64  <= n:
            XE, X, A = get_representation_input(cid_all[i*64:(i+1)*64])
            representations = get_fingerprint_fct([np.array(XE), np.array(X),np.array(A)])[0]
            df["GNN FP"][i*64:(i+1)*64] = list(representations[:, :52])
        else:
            print(i)
            XE, X, A = get_representation_input(cid_all[-64:])
            representations = get_fingerprint_fct([np.array(XE), np.array(X),np.array(A)])[0]
            df["GNN FP"][-64:] = list(representations[:, :52])
        i += 1
        
    ### set all GNN FP-entries with no input matrices to np.nan:
    all_X_matrices = os.listdir(input_data_folder)
    for ind in df.index:
        if prefix +str(ind) +"_X.npy" not in all_X_matrices:
            df["GNN FP"][ind] = np.nan
    return(df)

In [16]:
#Calculating the GNN representations
train_with_rep = get_substrate_representations(df = train_df, training_set = True, 
                                                      get_fingerprint_fct = get_fingerprint_fct)
test_with_rep = get_substrate_representations(df = test_df, training_set = False, 
                                                     get_fingerprint_fct = get_fingerprint_fct)

#Saving the DataFrames:
train_with_rep.to_pickle(join(datasets_dir, "splits", "training_data.pkl"))
test_with_rep.to_pickle(join(datasets_dir, "splits", "test_data.pkl"))

25
7
